In this blog post, we will have a look at how can we build AI chat with youtube videos. 

The main problem that I wanted to solve was, sometimes I need to watch a long videos to get the information that I need. So, I thought why not build a chatbot that can help me with that. This would not only save my time but also help me to get the information that I need quickly.

Checkout this blog in video 

[![AI Workshop - Chat with videos](https://img.youtube.com/vi/ISCLsXS9Sns/0.jpg)](https://www.youtube.com/watch?v=ISCLsXS9Sns "AI Workshop - Chat with videos")

Lets install the pytubefix library. We will be using this library to get the audio from the video.


In [7]:
!pip install pytubefix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


I have choosen a video that explains the concept of pointers in C language. I will be using this video to build the chatbot.

Checkout the video here

[![#23 C Pointers | C Programming For Beginners](https://img.youtube.com/vi/KGhacRRMnDw/0.jpg)](https://www.youtube.com/watch?v=KGhacRRMnDw "#23 C Pointers | C Programming For Beginners")

Lets download the audio from the video.

In [8]:
from pytubefix import YouTube
yt = YouTube("https://www.youtube.com/watch?v=KGhacRRMnDw")

In [12]:
ys = yt.streams.get_audio_only()
ys.download()

'/Users/rajesh/Documents/Work/personal/ai/notebooks/video-chat/23 C Pointers  C Programming For Beginners.mp4'

Now, lets install the openai library. We will be first transcribing the audio to text using the openai library and then use the LLM to generate coherent responses.

In [13]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 13.3 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Using cached jiter-0.5.0-cp311-cp311-macosx_11_0_arm64.whl (299 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.23.4-cp311-cp311-macosx_11_0_arm64.whl (1.8 MB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In the next step, I am setting up the openai api key, in the jupyter notebook. If you are not using jupyter notebook, you can set the api key in the environment variable.

In [15]:
import getpass
openai_key = getpass.getpass("Enter your OpenAI key: ")

!export OPENAI_API_KEY=$openai_key

Now, we do the transcription of the audio to text.

In [17]:
from openai import OpenAI

client = OpenAI(api_key=openai_key)

audio_file = open("./23 C Pointers  C Programming For Beginners.mp4", "rb")
transcription = client.audio.transcriptions.create(model="whisper-1", file=audio_file)
transcription = transcription.text

What's up guys? Welcome back to this series on C programming. In this video, we'll learn about pointers in C. More specifically, we'll learn to work directly with computer memory address with the help of pointers. So let's get started. Pointer is one of the main features that make C programming so powerful. It allows us to work directly with the computer memory. Before we learn about pointers, let's first learn about addresses. In C programming, whenever we declare a variable, a space will be allocated in the memory for the variable and C allows us to access the address of the variable. We use the ampersand symbol with the variable name to access the memory address. Let's see an example. You might be familiar with the basic structure of C program. Now I'll create an integer variable, age, so int age, and I'll assign value 25 to this. Now let's use ampersand symbol to access the address where the age variable is stored. So printf %p, ampersand age. Here I have used %p format specifier t

Now that we have transcribed the audio to text, we need to split the text into smaller chunks. This is because with most of the LLMs we need to keep the context to a certain limit. So, we will split the text into smaller chunks and then use the LLM to generate responses. We will use `nltk` library to split the text into smaller chunks.

In [19]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 21.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import nltk.data
nltk.download('punkt_tab')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rajesh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Using the tokenizer, we will split the text into smaller chunks.

In [23]:
lines = tokenizer.tokenize(transcription)

Lets group the sentences into smaller chunks. Something like 5 sentences in each chunk.

In [24]:
# group the lines array by combining 5 lines into a single string
grouped_lines = []
for i in range(0, len(lines), 5):
    grouped_lines.append(" ".join(lines[i : i + 5]))

Next, we will install ChromaDB, a popular vector database, that is very convinient to use with python. We will be using in-memory mode with ChromaDB.

In [26]:
!pip install chromadb

  Using cached chromadb-0.5.7-py3-none-any.whl (599 kB)
  Using cached build-1.2.2-py3-none-any.whl (22 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-macosx_11_0_arm64.whl (185 kB)
  Using cached fastapi-0.115.0-py3-none-any.whl (94 kB)
  Using cached uvicorn-0.30.6-py3-none-any.whl (62 kB)
  Using cached numpy-2.1.1-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Using cached posthog-3.6.6-py2.py3-none-any.whl (54 kB)
  Using cached onnxruntime-1.19.2-cp311-cp311-macosx_11_0_universal2.whl (16.8 MB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl (63 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl (18 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.48b0-py3-none-any.whl (11 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl (110 kB)
  Using cached tokenizers-0.20.0-cp311-cp311-macosx_11_0_arm64.whl (2.5 MB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl (17 kB)
  Using

Now that we have installed the ChromaDB, let initialize the database.

In [27]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
collection = client.create_collection("all-my-documents")

Let's feed the text to ChromaDB. ChromaDB will convert the text to vectors and store them in the database. Although, ChromaDB uses a smaller transformer model, it is still very powerful and can be used for a lot of NLP tasks. But for production use, you might want to use a bigger transformer model.

In [29]:
grouped_lines_metadata = [{"text": line} for line in grouped_lines]
# use the index of the line in the grouped_lines array as the document ID
grouped_lines_ids = [str(i) for i in range(len(grouped_lines))]

In [30]:
collection.add(
    documents=grouped_lines,  # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=grouped_lines_metadata,  # filter on these!
    ids=grouped_lines_ids,  # unique for each doc
)

Let's now define a user query and get the response from the ChromaDB.

In [42]:
question = "how to read a pointers value?"

In [43]:
results = collection.query(
    query_texts=[question],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

In [44]:
results
import json
context = json.dumps(results["documents"], indent=2)

Let's now download LangChain, a library that can be used to generate responses from the LLMs. 

In [45]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


Also install the openai version of langchain.

In [36]:
!pip install langchain-openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.2 MB/s eta 0:00:00
  Using cached tiktoken-0.7.0-cp311-cp311-macosx_11_0_arm64.whl (907 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


Lets now prompt the LLM with the user query and get the response. We are using a very basic prompt here. You can use more complex prompts to get better responses.

In [46]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI(api_key=openai_key)
prompt = PromptTemplate.from_template(
    """
    You are a C programming instructor. you can only answer from the context that is given to you.
    Question: {question}
    
    Context: {context}
    
    Based on the the question and the context, answer the question. Do not provide any information that is not present in the context.
"""
)

chain = prompt | llm
chain.invoke({"question": question, "context": context})

"    \n    To read a pointer's value, you need to use the asterisk symbol before the pointer variable name. This will give you the value stored at the memory address pointed by the pointer."

## Conclusion

In this blog post, we saw how can we build a chatbot that can help us with the information in the videos. We used the openai library to transcribe the audio to text and then used the LLM to generate responses. We also used the ChromaDB to store the text and get the responses. This is a very basic implementation of the chatbot. You can use more complex prompts and models to get better responses.